# in this file we will try and implement the CNN_TF algorithm 

In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score

In [ ]:
class CustomCNN:
    def __init__(self, input_shape=(20, 20), num_classes=10, learning_rate=0.01):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.initialize_weights()

    def initialize_weights(self):
        """ Xavier initialization for better training stability """
        self.conv1_filters = np.random.randn(3, 3, 1, 32) * np.sqrt(2 / (3 * 3 * 1))
        self.conv1_bias = np.zeros((32,))
        
        self.conv2_filters = np.random.randn(3, 3, 32, 64) * np.sqrt(2 / (3 * 3 * 32))
        self.conv2_bias = np.zeros((64,))
        
        self.fc1_input_size = 8 * 8 * 64
        self.fc1_weights = np.random.randn(128, self.fc1_input_size) * np.sqrt(2 / self.fc1_input_size)
        self.fc1_bias = np.zeros((128,))

        self.fc2_weights = np.random.randn(self.num_classes, 128) * np.sqrt(2 / 128)
        self.fc2_bias = np.zeros((self.num_classes,))

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward_pass(self, X):
        """ Forward propagation through the network """
        X = X.reshape(-1, 20, 20, 1)

        # Conv Layer 1
        conv1_output = np.zeros((X.shape[0], 18, 18, 32))
        for i in range(18):
            for j in range(18):
                region = X[:, i:i+3, j:j+3, :]
                conv1_output[:, i, j, :] = np.tensordot(region, self.conv1_filters, axes=([1, 2, 3], [0, 1, 2])) + self.conv1_bias

        conv1_output = self.relu(conv1_output)

        # Conv Layer 2
        conv2_output = np.zeros((conv1_output.shape[0], 16, 16, 64))
        for i in range(16):
            for j in range(16):
                region = conv1_output[:, i:i+3, j:j+3, :]
                conv2_output[:, i, j, :] = np.tensordot(region, self.conv2_filters, axes=([1, 2, 3], [0, 1, 2])) + self.conv2_bias

        conv2_output = self.relu(conv2_output)

        # Max Pooling (2x2)
        pooled_output = conv2_output[:, ::2, ::2, :]
        flat_output = pooled_output.reshape(pooled_output.shape[0], -1)  # Shape (batch_size, 4096)

        # Fully Connected Layer 1
        fc1_output = self.relu(np.dot(flat_output, self.fc1_weights.T) + self.fc1_bias)

        # Fully Connected Layer 2 (Output)
        logits = np.dot(fc1_output, self.fc2_weights.T) + self.fc2_bias
        output = self.softmax(logits)

        return output, flat_output, fc1_output

    def compute_loss(self, y_true, y_pred):
        """ Cross-entropy loss """
        m = y_true.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_true])
        return np.sum(log_likelihood) / m

    def backward_pass(self, X, y_true, y_pred, flat_output, fc1_output, learning_rate):
        """ Backpropagation and weight updates using SGD """
        m = y_true.shape[0]

        # Compute gradients
        d_logits = y_pred
        d_logits[range(m), y_true] -= 1
        d_logits /= m

        # Gradients for fully connected layer 2
        d_fc2_weights = np.dot(d_logits.T, fc1_output)
        d_fc2_bias = np.sum(d_logits, axis=0)

        d_fc1_output = np.dot(d_logits, self.fc2_weights)

        # Gradients for fully connected layer 1
        d_fc1_output *= self.relu_derivative(fc1_output)
        d_fc1_weights = np.dot(d_fc1_output.T, flat_output)
        d_fc1_bias = np.sum(d_fc1_output, axis=0)

        # Weight updates using simple SGD
        self.fc2_weights -= learning_rate * d_fc2_weights
        self.fc2_bias -= learning_rate * d_fc2_bias

        self.fc1_weights -= learning_rate * d_fc1_weights
        self.fc1_bias -= learning_rate * d_fc1_bias

    def fit(self, X_train, y_train, epochs=10, batch_size=128):
        """ Training the model """
        start_time = time.time()
        for epoch in range(epochs):
            for i in range(0, X_train.shape[0], batch_size):
                X_batch = X_train[i:i+batch_size]
                y_batch = y_train[i:i+batch_size]

                # Forward Pass
                y_pred, flat_output, fc1_output = self.forward_pass(X_batch)

                # Compute Loss
                loss = self.compute_loss(y_batch, y_pred)

                # Backpropagation
                self.backward_pass(X_batch, y_batch, y_pred, flat_output, fc1_output, self.learning_rate)

                if i == 0:  # Print loss for the first batch of each epoch
                    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

        end_time = time.time()
        print(f"Training completed in {round(end_time - start_time, 2)} seconds.")

    def predict(self, X):
        y_pred, _, _ = self.forward_pass(X)
        return np.argmax(y_pred, axis=1)

In [14]:
# Step 1: Load Preprocessed Data (Cropped CSV)
train_df = pd.read_csv("../2PREPROCESSING/Processed_CSV/cropped_train.csv")
test_df = pd.read_csv("../2PREPROCESSING/Processed_CSV/cropped_test.csv")

# Separate Features (X) and Labels (y)
X_train, y_train = train_df.iloc[:, 1:].values, train_df.iloc[:, 0].values
X_test, y_test = test_df.iloc[:, 1:].values, test_df.iloc[:, 0].values

In [15]:
# Step 2: Train and Evaluate the Implemented CNN
custom_cnn = CustomCNN(input_shape=(20, 20), num_classes=10)

start_time = time.time()
custom_cnn.fit(X_train, y_train, epochs=10, batch_size=128)
end_time = time.time()

y_pred_custom_cnn = custom_cnn.predict(X_test)

# Compute Accuracy
accuracy = accuracy_score(y_test, y_pred_custom_cnn)
print(f"\nTest Accuracy: {accuracy:.4f}")
print(f"Training Time: {round(end_time - start_time, 2)} seconds")

Epoch 1, Loss: 2.5359
Epoch 2, Loss: 0.2803
Epoch 3, Loss: 0.2191
Epoch 4, Loss: 0.1882
Epoch 5, Loss: 0.1666
Epoch 6, Loss: 0.1489
Epoch 7, Loss: 0.1342
Epoch 8, Loss: 0.1220
Epoch 9, Loss: 0.1108
Epoch 10, Loss: 0.1009
Training completed in 349.07 seconds.

Test Accuracy: 0.9634
Training Time: 349.07 seconds


# this custom algorithm has accuracy of 96.34%